# Input Arguments -----------------------------------------------------------------------

In [1]:
outdir_root = '/Users/josebolorinos/Google Drive/Codiga Center/Charts and Data/Monitoring Reports'
hmi_path = '/Users/josebolorinos/Google Drive/Codiga Center/lbre-cr2c-col.stanford.edu/Reactor Feeding - Raw_20180917120201.csv'
start_dt_str = '7-14-18'
hmi_start_dt_str = '9-8-18'
end_dt_str = '9-14-18'

# Load Modules

In [2]:
# Utilities
import os 
from datetime import datetime as dt
from datetime import timedelta

# CR2C
import cr2c_labdata as pld
import cr2c_hmidata as hmi
import cr2c_fielddata as fld
import cr2c_validation as val

# Create Directories

In [3]:
outdir = os.path.join(outdir_root,'Monitoring Report ' + end_dt_str)

if not os.path.exists(outdir): 
    
    os.mkdir(outdir)
    
labOutdir = os.path.join(outdir,'Lab Data')

if not os.path.exists(labOutdir): 
    
	os.mkdir(labOutdir)
    
HMIOutdir = os.path.join(outdir,'Operational Data')

if not os.path.exists(HMIOutdir): 
    
	os.mkdir(HMIOutdir)
    
valOutdir = os.path.join(outdir,'Validation')

if not os.path.exists(valOutdir): 
    
	os.mkdir(valOutdir)

# Set Date Variables

In [4]:
mo6_start_dt = dt.strptime(end_dt_str,'%m-%d-%y') - timedelta(days = 180)
mo6_start_dt_str = dt.strftime(mo6_start_dt,'%m-%d-%y')
mo1_start_dt = dt.strptime(end_dt_str,'%m-%d-%y') - timedelta(days = 30)
mo1_start_dt_str = dt.strftime(mo1_start_dt,'%m-%d-%y')

# Update with latest Lab and Field Data

In [5]:
# Initialize lab data class
cr2c_lr = pld.labrun() 
# Lab Data
cr2c_lr.process_data()

# Field Data
fld.process_data()

/anaconda3/envs/cr2c-monitoring/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


# Update with latest HMI Data

In [6]:
# Initialize HMI class
hmi_run = hmi.hmi_data_agg(
    hmi_start_dt_str, # Start of date range we want summary data for 
    end_dt_str, # End of date range we want summary data for)
    hmi_path = hmi_path
)

# Hourly averages
hmi_run.run_report(
    [1]*14, # Number of hours/minutes we want to average over
    ['hour']*14, # Type of time period (can be "hour" or "minute")
    [
        'FT700','FT704','FT200','FT202','FT305',
        'AIT302','AT304','AT310','AT203','AT305',
        'DPIT300','DPIT301','PIT700','PIT704'
    ], # Sensor ids that we want summary data for (have to be in HMI data file obviously)
    [
        'gas','gas','water','water','water',
        'tmp','temp','temp','ph','ph',
        'dpi','dpi','pressure','pressure'
    ] # Type of sensor (case insensitive, can be water, gas, pH, conductivity, temp, or tmp
)

# Minute averages (for sensors we are validating and membrane parameters)
hmi_run.run_report(
    [1]*8, # Number of hours/minutes we want to average over
    ['minute']*8, # Type of time period (can be "hour" or "minute")
    ['FT305','AIT302','AT203','AT305','DPIT300','DPIT301','PIT700','PIT704'], # Sensor ids that you want summary data for (have to be in HMI data file obviously)
    ['water','tmp','ph','ph','dpi','dpi','pressure','pressure'] # Type of sensor (case insensitive, can be water, gas, pH, conductivity, temp, or tmp
)


Getting aggregated data for FT700 (1HOUR)...
Getting aggregated data for FT704 (1HOUR)...
Getting aggregated data for FT200 (1HOUR)...
Getting aggregated data for FT202 (1HOUR)...
Getting aggregated data for FT305 (1HOUR)...
Getting aggregated data for AIT302 (1HOUR)...
Getting aggregated data for AT304 (1HOUR)...
Getting aggregated data for AT310 (1HOUR)...
Getting aggregated data for AT203 (1HOUR)...
Getting aggregated data for AT305 (1HOUR)...
Getting aggregated data for DPIT300 (1HOUR)...
Getting aggregated data for DPIT301 (1HOUR)...
Getting aggregated data for PIT700 (1HOUR)...
Getting aggregated data for PIT704 (1HOUR)...
Getting aggregated data for FT305 (1MINUTE)...
Getting aggregated data for AIT302 (1MINUTE)...
Getting aggregated data for AT203 (1MINUTE)...
Getting aggregated data for AT305 (1MINUTE)...
Getting aggregated data for DPIT300 (1MINUTE)...
Getting aggregated data for DPIT301 (1MINUTE)...
Getting aggregated data for PIT700 (1MINUTE)...
Getting aggregated data for 

# Get Lab Data Plots

In [7]:
# Create and output charts
pld.get_lab_plots(
	# Plot start date
	start_dt_str,
	# Plot end date
	end_dt_str,
	# List of monitoring data types to produce charts for (correspond to tabs on gsheets workbook)
	['PH','ALKALINITY'], 
	# Variable to break down into panels according to
	'Stage',
	# Stages to Subset to
	['Microscreen','MESH','AFBR'],
	outdir = labOutdir
)
pld.get_lab_plots(
	# Plot start date
	start_dt_str,
	# Plot end date
	end_dt_str,
	# List of monitoring data types to produce charts for (correspond to tabs on gsheets workbook)
	['COD','TSS'], 
	# Variable to break down into panels according to
	'Type',
	# Stages to Subset to
	stage_sub = ['AFBR'],
	type_sub = ['TSS','VSS','Total','Soluble','Particulate'],
	outdir = labOutdir,
	opfile_suff = 'AFBR'
)
pld.get_lab_plots(
	# Plot start date
	start_dt_str,
	# Plot end date
	end_dt_str,
	# List of monitoring data types to produce charts for (correspond to tabs on gsheets workbook)
	['BOD'], 
	# Variable to break down into panels according to
	'Stage',
	# Stages to Subset to
	['Microscreen','MESH'],
	outdir = labOutdir, 
	opfile_suff = 'MS_MESH'
)
pld.get_lab_plots(
	# Plot start date
	mo6_start_dt_str,
	# Plot end date
	end_dt_str,
	# List of monitoring data types to produce charts for (correspond to tabs on gsheets workbook)
	['AMMONIA'], 
	# Variable to break down into panels according to
	'Stage',
	# Stages to Subset to
	['Microscreen','MESH'],
	outdir = labOutdir,
	opfile_suff = 'MS_MESH'
)
pld.get_lab_plots(
	# Plot start date
	mo6_start_dt_str,
	# Plot end date
	end_dt_str,
	# List of monitoring data types to produce charts for (correspond to tabs on gsheets workbook)
	['VFA'], 
	# Variable to break down into panels according to
	'Type',
	# Stages to Subset to
	outdir = labOutdir
)
# Get wide tables
cr2c_lr.summarize_tables(end_dt_str, 240, add_time_el = True, outdir = labOutdir)


# Get HMI Data Plots

In [8]:
hmi_run.get_tmp_plots(
	start_dt_str,
	end_dt_str,
	outdir = HMIOutdir
)
hmi_run.get_temp_plots(
	end_dt_str, 
	plt_colors =['c','m'],
	outdir = HMIOutdir
)
hmi_run.get_feed_sumst(
	'GAS',
	['plot','table'],
	start_dt_str,
	end_dt_str,
	sum_period = 'DAY',
	plt_type = 'bar', 
	plt_colors = ['#90775a','#eeae10'],
	ylabel = 'Biogas Production (L/day)',
	outdir = HMIOutdir
)
hmi_run.get_feed_sumst(
	'WATER',
	['plot','table'],
	start_dt_str,
	end_dt_str,
	sum_period = 'DAY',  
	plt_type = 'bar', 
	plt_colors = ['#8c9c81','#7fbfff'],
	ylabel = 'Reactor Feeding (Gal/Day)',
	outdir = HMIOutdir
)

# Get Validation Plots

In [9]:
# Initialize validation class
cr2c_vl = val.cr2c_validation(outdir = valOutdir, hmi_path = hmi_path)

# Get COD balance and biotech parameters (with plots)
cr2c_vl.get_cod_bal(end_dt_str, 8, plot = True)
cr2c_vl.get_biotech_params(end_dt_str, 8, plot = True)

# Get validation plots
cr2c_vl.instr_val(
    valtypes = ['PH','PH'],
    start_dt_str = mo1_start_dt_str,
    end_dt_str = end_dt_str,
    hmi_elids = ['AT203','AT305'],
    ltypes = ['PH','PH'],
    lstages = ['Microscreen','AFBR']
)
cr2c_vl.instr_val(
    valtypes = ['DPI','DPI','PRESSURE','PRESSURE'],
    start_dt_str = mo1_start_dt_str,
    end_dt_str = end_dt_str,
    hmi_elids = ['DPIT300','DPIT301','PIT700','PIT704'],
    fld_varnames = [('Before Pump: R300','After Pump: R300'),('Before Pump: R301','After Pump: R301'),'Manometer Pressure: R300','Manometer Pressure: R301']
)

/anaconda3/envs/cr2c-monitoring/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/josebolorinos/Google Drive/Codiga Center/Code/cr2c-monitoring/cr2c_fielddata.py:110: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  join = 'outer'
